In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pairprog.objectstore import * 
from pairprog.assistant import * 
from pairprog.assistant  import logger as as_logger

In [3]:
import logging
logging.basicConfig()
as_logger.setLevel(logging.DEBUG)

In [4]:
rc = ObjectStore.new(name='barker_minio', bucket='agent')

In [5]:
import typesense
ts = typesense.Client(
            {
                "api_key": "xyz",
                "nodes": [{"host": "barker", "port": "8108", "protocol": "http"}],
                "connection_timeout_seconds": 1,
            }
        )

tool = PPTools(ts, rc.sub('pptools'), Path('/Volumes/Cache/scratch'))

assis = Assistant(tool, cache=rc)

messages = [
    {"role": "user", "content": 'tell me a joke'}
]

def dump(o):
    import json 
    print( json.dumps(o.model_dump(), indent=2))


In [6]:
for t in tool.specification():
    print(t['function']['name'])

execute_code
get_document
read
search_documents
shell
store_document
write


In [7]:
def get_sessions():
    
    sessions = set(e.split('/')[0] for e in list(rc.sub('session').list()))
    return list(reversed(sorted(sessions)))

def get_last_src():

    last_session  = get_sessions()[0]
    src = rc.sub('session/'+last_session)
    return src

def dump_sessions():
    last_session  = get_sessions()[-3]
    src = rc.sub('session/'+last_session)
    for e in src['responses']:
        print(e)
        print('-----')

src = get_last_src()

for e in src['messages']:
    print(e)
    print('-----')


{'role': 'user', 'content': 'what time is it'}
-----
{'content': '', 'function_call': None, 'role': 'assistant', 'tool_calls': [{'index': 0, 'id': 'call_zYr5Exnr0noGOo2eOj9BIYEe', 'function': {'arguments': '{"command":"date +%T"}', 'name': 'shell'}, 'type': 'function'}]}
-----
{'tool_call_id': 'call_zYr5Exnr0noGOo2eOj9BIYEe', 'role': 'tool', 'name': 'shell', 'content': '20:29:04\n'}
-----


In [8]:
if False:
    for k in src.list('loop/post/'):
        o = src[k]
        print(json.dumps(o, indent=2))
    

def dump_last_tool_calls():
    for k in src.list('loop/tool_call'):
        o = src[k]
        print(o)
        try:
            
            a = o['args']
            args = json.loads(a)
            print(args['code'])
        except:
            pass
    
        try:
            
            a = o['args']
            args = json.loads(a)
            print(args['path'])
        except:
            pass
        
        print('========')

In [84]:
messages = [
    { "role": "user", "content": "Why is the weather colder near the poles than near the equator? Explain it to me, then save "+
    "your answer to the file weather.txt"}
]

assis.messages = messages

g = assis.client.chat.completions.create(
                messages=assis.messages,
                tools=assis.func_spec,
                model=assis.model,
                stream=True,
                timeout=10
            )


chunk, responses = assis.process_streamed_response(g, call_back=assis.print_content_cb)


The temperature difference between the poles and the equator is primarily due to the angle of the Sun's rays and the distribution of solar energy. Here's an explanation:

1. Angle of Sun's Rays: Near the equator, the Sun's rays are more direct, meaning they cover a shorter distance through the atmosphere. This results in a higher concentration of solar energy and greater heating. At the poles, the Sun's rays are at a much lower angle, leading to a more scattered distribution of solar energy and less heating.

2. Length of Daylight: Near the poles, the length of daylight varies significantly throughout the year, with extensive periods of continuous darkness during the winter months. This leads to longer and harsher winters with minimal heating from the Sun.

3. Atmospheric Circulation: The Earth's rotation and the tilt of its axis create global wind patterns. Near the equator, warm air rises and moves toward the poles, while cold air sinks and flows towards the equator. This results in 

{'content': 'The temperature difference between the poles and the equator is primarily due to the angle of the Sun\'s rays and the distribution of solar energy. Here\'s an explanation:\n\n1. Angle of Sun\'s Rays: Near the equator, the Sun\'s rays are more direct, meaning they cover a shorter distance through the atmosphere. This results in a higher concentration of solar energy and greater heating. At the poles, the Sun\'s rays are at a much lower angle, leading to a more scattered distribution of solar energy and less heating.\n\n2. Length of Daylight: Near the poles, the length of daylight varies significantly throughout the year, with extensive periods of continuous darkness during the winter months. This leads to longer and harsher winters with minimal heating from the Sun.\n\n3. Atmospheric Circulation: The Earth\'s rotation and the tilt of its axis create global wind patterns. Near the equator, warm air rises and moves toward the poles, while cold air sinks and flows towards the 

In [101]:
from copy import deepcopy



cmessages = deepcopy(messages)
d = deepcopy(chunk).choices[0].delta
cmessages.append({'role':'assistant', 'content':d.content})
cmessages.extend(call_function(assis, chunk))
cmessages

Calling tool 'write'


[{'role': 'user',
  'content': 'Why is the weather colder near the poles than near the equator? Explain it to me, then save your answer to the file weather.txt'},
 {'role': 'assistant',
  'content': 'The temperature difference between the poles and the equator is primarily due to the angle of the Sun\'s rays and the distribution of solar energy. Here\'s an explanation:\n\n1. Angle of Sun\'s Rays: Near the equator, the Sun\'s rays are more direct, meaning they cover a shorter distance through the atmosphere. This results in a higher concentration of solar energy and greater heating. At the poles, the Sun\'s rays are at a much lower angle, leading to a more scattered distribution of solar energy and less heating.\n\n2. Length of Daylight: Near the poles, the length of daylight varies significantly throughout the year, with extensive periods of continuous darkness during the winter months. This leads to longer and harsher winters with minimal heating from the Sun.\n\n3. Atmospheric Circul

In [132]:
assis = Assistant(tool, cache=rc)
assis.run('Why is the weather colder near the poles than near the equator? '+
          'Explain it to me, then save your answer to the file weather.tx')


Sure, I can help explain that. The Earth's shape and its axis tilt are two key factors that influence the temperature variations between the poles and the equator. In simple terms, the equator receives more direct sunlight and heat from the sun, while the poles receive sunlight at a much lower angle.

At the equator, the sun's rays strike the Earth's surface almost perpendicularly, providing greater heat and energy. This proximity to the sun leads to higher temperatures in these regions.

Conversely, near the poles, the sun's rays strike the Earth's surface at a sharp angle, causing the sunlight to be spread out over a larger area. This results in lower temperatures, as the same amount of sunlight and heat is dispersed over a larger area.

I will save this explanation to the file "weather.txt" for you.Calling tool 'write'
The explanation has been saved to the file "weather.txt." If you have any further questions or if there's anything else I can assist you with, feel free to ask!

{'role': 'assistant',
 'content': 'The explanation has been saved to the file "weather.txt." If you have any further questions or if there\'s anything else I can assist you with, feel free to ask!'}

In [ ]:
assis.run('show me what you wrot

In [131]:
self = assis
messages = deepcopy(self.request_messages())
del messages[2]['function_call']
g = self.client.chat.completions.create(
                messages=messages,
                tools=self.tools.specification(),
                model=self.model,
                stream=True,
                timeout=10
            )